## 决策树
### Splitting Datasets one feature at a time: Decision Tree
决策树分类器就像带有终止块的流程图，终止块表示分类结果。

### 1. 决策树学习基本算法

**输入**：训练集 $ D = \{(\textbf{x}_1, y_1), (\textbf{x}_2, y_2), ..., (\textbf{x}_m, y_m)\} $
         , 特征集 $ A = \{a_1, a_2, .., a_n\} $
         
**算法函数**: `TreeGenerate(D, A)`
```python
    def treeGenerate(D, A):
        node = generateNode
        if (D 中样本同属于一类 C) then: (情况 1)
            将 node 标记为 C 类叶结点，
            return node
        if (A = ∅ OR D 中的样本在 A 上的所有特征的取值都相同) then: (情况 2)
            将 node 标记为叶结点，其类别标记为 D 中样本数最多的类 (即投票);
            return node
        
        从 A 中选择最优的划分特征 a*;
        
        for a* 的每一个值 vi do:
            为 node 生成一个分支；令 Dv 为 D 在a* 取值为 vi 时的样本子集。
            if Dv = ∅ ：
                将新的分支结点标记为叶子结点，基类别是 D 中样本最多的类 (投票) (情况 3)
                return
            else:
                以 TreeGenerate(Dv, A - {a*}) 为分支节点进行递归创建。
            end if
        end for            
```

**输出**: 以 node 为根结点的一棵决策树。

可以看出来，生成决策树是一个递归过程。有三种情况会导致递归返回：

1. 当前结点包含的样本全属于同一个类别，无需要划分。
2. 当前特征集为空，或者所有样本在所有属性上的取值相同，无法划分。 （以当前结点的样本投票）
3. 当前结点包含的样本集合为空，不能划分。（以父节点的样本投票）

### 2. 划分选择

决策树最关键的问题是怎么选择最优的划分。一般而言，随着划分过程的不断进行，我们希望决策树的分支节点所包含的样本尽可能属于同一类别，即结点的纯度(purity) 越来越高。
#### 2.1 使用信息增益划分

熵是度量数据集中数据无序度的一种方法。
##### 2.1.1 熵与信息增益(Entropy and Information Gain) 
#####  定义
>通常，一个信源发送出什么符号是不确定的，衡量它可以根据其出现的概率来度量。

>概率大，出现机会多，不确定性小；反之就大。不确定性函数 f 是概率 P 的单调递降函数，即概率越大，不确定性越小；反之亦然。

>假设两个符号(信源)出现的概率是 $p_1, p_2$, 不确定函数为  $ f $ ，则两个独立符号所产生的不确定性应等于各自不确定性之和，即$f(p_1，p_2)=f(p_1)+f(p_1)$
这称为可加性。同时满足这两个条件的函数$f$是对数函数，即 
>>$f(p_i) = \log\frac1p_i = -\log{p_i}$, 式中对数一般取2为底，单位为比特。

> 所以我们把符号 $x_i$ 的**自信息**定义为：$l(x_i) = -\log_2{p(x_i)}$, 其中 $ p(x_i) $ 是符号 $x_i$ 出现的概率。

> 这时，信源的平均不确定性应当为单个符号不确定性$-log_2{P_i}$的统计平均值（E），可称为*信息熵* ([香农熵, Shannon entropy](https://youtu.be/R4OlXb9aTvQ))，即香农熵是样本空间内所有符号信息的 **期望**。
>> $ H = - \sum_{i=1}^n p(x_i)\log_2{p(x_i)} $

熵越大，则说明变量的不确定性越大，无序度越大。而熵越小，则表明数据集的纯度越高。

##### 信息增益
> 信息增益是熵的减少或者是数据无序度的减少。


假定样本集合 D 中，第 k 类样本所占的比例为 $p_k （k = 1, 2, ...|y|）$, 则 D 的信息熵为:

$$
Ent(D) = -\sum_{k = 1}^{|y|} {p_k} log{p_k}
$$

Ent(D) 越小，其纯度越高。越大则越无序。

假设 **离散** 属性 $a$ 有 V 个可能的取值 $\{a^1, a^2, ..., a^V \}$。 若使用 a 来对样本集 D 进行划分，则会产生 V 个分支节点。假设 $D_v$ 表示在 a 在取值 $a^v$ 时的所有样本，那么我们可以计算出在使用 a 对样本集 D 进行划分时所获得的 『信息增益(information gain)』。

$$
    Gain(D, a) = Ent(D) - \sum_{v=1}^{V} \frac{|D_v|}{|D|}Ent(D_v)
$$

一般而言，信息增益越大，则意味着使用属性 a 来进行划分所获得的『纯度』提升越大。所以划分属性的选择如下：

$$
a_* = arg \space max \space Gain(D, a), a \in A
$$

著名的 **ID3 (Iterative Dichotomiser)** 就是以信息增益来进行划分的。

#### 2.2 使用增益率来进行划分
在使用信息增益来进行划分时，如果样本的编号作为一个特性，那么该特征的信息增益一定是最大的。那么使用该特征进行划分，那就会生成和样本集一样多的分支，而这些分支结点只有一个样本，显然纯度已经最大了，不能进行划分了。这样决策树显然不具有泛化能力了，无法对新的样本进行有效的预测。

实际上，信息增益准则对可取值数目较多的属性有所偏好，为了减少这种偏好，我们可以使用 **增益率(gain ratio)** 来选择最优划分属性。增益率的定义为：

$$
Gain\_ratio(D, a) = \frac{Gain(D, a)}{IV(a)}
$$

其中：

$$
IV(a) = -\sum_{v=1}^{V}\frac{|D^v|}{|D|}log(\frac{|D^v|}{|D|})
$$

IV, 称为**固有值 (intrinsic value)**。属性 a 取值的可能数目越多则 $IV(a)$ 的值就越大。增益率就是让其与 $IV(a)$ 成反比。

著名的 **C4.5 (Iterative Dichotomiser)** 就是以增益率来进行划分的。



**Note**： **信息增益**偏好取值多的属性，而**信息增益率**准则又偏好于取值少的属性。所以实现中，可以用一个启发式的搜索，即先从候选划分属性中找出信息增益高于平均水平的属性，现从中选择增益率最好的。


#### 2.3 使用基尼指数来划分
**CART** 决策树使用『基尼指数』(Gini Index) 来选择划分属性。数据集 D 的纯度可以用基尼值来度量：

$$
Gini(D) = \sum_{k=1}^{|y|}\sum_{k' \neq k} p_k p_{k'} = \sum_{k=1}^{|y|}p_k(1 - p_k)  =  1 - \sum_{k=1}^{|y|}p_k^2
$$

上式一个简单的推导，注意: $ \sum_{k=1}^{|y|}p_k = 1$.

\begin{align}
Gini(D) &= \sum_{k=1}^{|y|}\sum_{k' \neq k} p_k p_{k'}\\
&= p_1 \sum_{k' \neq 1} p_k' + p_2 \sum_{k' \neq 2} p_k' + \cdots +  p_{|y|} \sum_{k' \neq |y|} p_k' \\
&= p_1(1 - p_1) + p_2(1 - p_2) + \cdots + p_{|y|}(1 - p_{|y|}) \\
&= \sum_{k=1}^{|y|}p_k(1 - p_k) \\
&= \sum_{k=1}^{|y|}(p_k - p_k^2) \\
&= \sum_{k=1}^{|y|}p_k - \sum_{k=1}^{|y|}p_k^2 \\
&= 1 - \sum_{k=1}^{|y|}p_k^2
\end{align}

直观来说，Gini(D) 反映了从数据集 D 中随机制取两个样本，其类别标记不一致的概率，因此， Gini(D) 越小，则数据集 D 的纯度越高。

而属性 a 的基尼指数定义为:

$$
Gini\_index(D, a) = \sum_{v=1}^{V} \frac{D^v}{D}Gini(D^v)
$$
选择最优的划分属性，则使用划分全基尼指数**最小**的属性。

$$
a_* = arg \space min \space Gini\_index(D, a), a \in A
$$

### 2.4 方差缩减（Variance Reduction) 

在 ML 书中一般只介绍了信息熵和 Gini。但是还有一个常用的划分方法那就是**方差缩减(Variance Reduciton)**。 

在 CART 中，当目标问题是回归问题时就会使用 Varaince Reduction 来生成回归树。因为应用其它划分方法需要先对特征值进行离散化才行。

$$
I_{V}(N)={\frac {1}{|S|^{2}}}\sum _{i\in S}\sum _{j\in S}{\frac {1}{2}}(x_{i}-x_{j})^{2}-\left({\frac {1}{|S_{t}|^{2}}}\sum _{i\in S_{t}}\sum _{j\in S_{t}}{\frac {1}{2}}(x_{i}-x_{j})^{2}+{\frac {1}{|S_{f}|^{2}}}\sum _{i\in S_{f}}\sum _{j\in S_{f}}{\frac {1}{2}}(x_{i}-x_{j})^{2}\right)
$$

上式中，$S, S_t, S_f$ 分别表示原始样本集，正(true)样本集，正(false)样本集。上面的公式只是写的复杂，其实就是元素之间相互减而已。最终是求是方差。

可以简化为下面的公式：

$$
I_V(N) = Var(S) - \big(\frac{|S_t|}{|S|}Var(S_t) + \frac{|S_f|}{|S|}Var(S_f)\big)
$$


CART 对回归树用方差缩减准则，而对分类树则用基尼指数(Gini index) 最小化准则，进行特征选择，生成二叉树。

### 3. 剪枝处理 (pruning)

剪枝(pruning) 是决策树防止『过拟合』的主要手段。因为有时分支太多，会降低树的泛化能力导到过拟合。有两种剪枝策略：**预剪枝(prepruning), 后剪枝(postpruning)**.

**预剪枝**是在构建决策树时，判断该划分是否提高了树的泛化能力，如果提升了则进行划分，如是没有提升则停止划分。

**后剪枝**是在决策树生成之后，通过自底向上的对**非叶子**节点进行考察，若将子树替换成叶子节点能提升泛化能力，则进行替换。

泛化能力的检测，需要我们使用验证集 (Validation Set）来对比剪枝前后的准确率。

### 4. 连续值属性处理

如果属性的取值是连续性的，那么连续性可以取值数目不再有限了。我们可以对连续值进行离散化。最简单的策略是采用 **二分法(bi-partion)** 对连续性进行处理。其策略如下：

假设样本集 D 和 连续性属性 a, 对 a 的 n 个连续值进行递增排序， 记为: $\{a^1, a^2, ..., a^n\}$。我们需要找到一个划分点 t , 将划分后的两个集合 $D_t^-, D_t^+$ 的纯度达到最高, 其中 $D_t^-$ 是值不大于 t 的样本集合，而 $D_t^+$ 是大于 t 的样本集合。我们可取的划分点候选集合定义为：

$$
T_a = \{\frac{a^i + a^{i + 1}} {2} \space | \space 1 \leqslant i \leqslant n - 1\}
$$

即把取两个相邻点的中间值作为划分点，若有 n 个取值，则有 $n -1$ 个划分点。然后我们从这个  $n - 1$ 个划分点找出一个最佳的划分点, 可以用离散值的试来考察候选划分点，如使用下面的式子：

$$
Gain(D, a) = \max_{t \in T_a} Gain(D, a, t) = \max_{t \in T_a} Ent(D) - \sum_{\lambda \in \{-,+\}}\frac{|D_t^\lambda|}{|D|}Ent(D_t^\lambda|)
$$

其中 Gain(D, a, t) 是 D 根据划分点 t 二分后的信息增益。

### 5. 缺失值处理

当样本集中的样本在属性 a 上的值缺失时，我们将面临两个问题。

1. 在样本值缺失时，怎么计算信息增益，来确定划分属性那？
2. 在样本值缺件时，那么该样本应该属于那个子节点那？

上面两个问题，我们需要计算属性 a 上所有可能的取值的 $a^v$ 在所有可能取值上的比例，即 $\frac{|\tilde{D}^v|}{|\tilde{D}|}$。其中 $\tilde{D}$ 是在属性 a 上无缺失值的集合。 即 $\tilde{D} \subset D$, $\tilde{D} $ 在属性 a 上无缺失值。

具体的参考周志华老师的《机器学习》。


### 6. 多变量决策树
多变量决策树(multivariate decision tree) 也称为 『斜决策树』(oblique decision tree)， 在此类决策树中非叶子节点不再是对单个属性，而是对多个属性的线性组合来进行划分。即每一个非叶子节点是一个形如  $\sum_{i=1}^{d} w_i a_i = t$ 的线性分类器, 即对 d 个属性找出一个合适的线性分类器，而不是找一个最优的划分属性。

### 构造决策树
Ex. 海洋生物数据，两个特征：

1. no surfacing; 
2. flippers

Order| No Surfacing| flippers | fish
---|---|---|---|
 1| 1 | 1| y
 2| 1 | 1| y
 3| 1 | 0| n
 4| 0 | 1| n
 5| 0 | 1| n
 
#### 1. 计算香农熵(熵越高，则混合的数据也就越多)
 \begin{align}
 H & = -p(x_y) * \log_2{p(x_y)}   -p(x_n) * \log_2{p(x_n)}
   \\ & = -(\frac{2}{5} *  log_2{\frac{2}{5}}) - -(\frac{3}{5} *  log_2{\frac{3}{5}})
   \\ &\approx 0.5288 + 0.4422 \\ &= 0.971
 \end{align}
 
#### 2. 按照获取最大信息增益的方法划分数据集(第一轮)
分别根据不同的特征来确定数据集的划分，用最大信息增益的特征来划分。
1. 以 no-surfacing 特征来尝试分类：
```
    feature = 'no surfacing',    value = 1 : [1, y], [1, y], [0, n]
                                 value = 0 : [1, no], [1, no]
```

则新的熵为 $h_1$ 与信息增益 $g_1$：
\begin{align}
h_1 &= \frac{3}{5}* (-\frac{2}{3} * \log_2{\frac{2}{3}}  -\frac{1}{3} * \log_2{\frac{1}{3}}) + \frac{2}{5} * (-\log_2{1})
\\ &\approx \frac{3}{5} * (0.39 + 0.528) \\ &= 0.5508
\\ \\ g_1 &= H - h_1 = 0.971 - 0.5508 = 0.4202
\end{align}

2. 以 flippers 第二个特征来尝试分类：
```
    feature = 'flippers',       value = 1: [1, y], [1,y], [0, n], [0, n]
                                value = 0: [1, no]
```
则新的熵为 $h_2$ 与信息增益 $g_2$:
\begin{align}
h_2 &= \frac{4}{5}(-\frac{1}{2} * \log_2{\frac{1}{2}} - \frac{1}{2} * \log_2{\frac{1}{2}}) + \frac{1}{5}(-\log_2{1})
\\ & = 0.8 
\\ \\ g_2 &= H - h_2 = 0.971 - 0.8 = 0.170951
\end{align}

根据最大的信息增益来划分数据集，即根据第一个特征来划分：
<img src="./decisionTrees01.svg" />
 

#### 2. 递归按照获取最大信息增益的方法划分数据集(第二轮)¶
即对还需要划分的子树进行划分，待划分的子树就只有右子树了。
其基本熵为：
\begin{align}
H = - \frac{2}{3} * log_2{\frac{2}{3}} - \frac{1}{3} * log_2{\frac{1}{3}} \approx 0.6365
\end{align}
该子树就只有一个特征值 flippers 了, 根据 flippers 来进行划分。
```
    features = flippers, value = 1: [1, y], [1, y]
                         value = 0: [0, n]
```
新的划分熵 $h_1$ 及信息增益 $g_1$:
\begin{align}
h_1 &= \frac{2}{3} * (-log_2{1}) + \frac{1}{3} * (-log_2{1}) = 0
\\ g_1 & = 0.6365 - 0 = 0.6365
\end{align}
无更多信息增益，可以直接划分。

#### 递归结束的条件
决策树递归结束的条件是：程序遍历完所有划分数据集的属性，或者每个分支下的所有实例都具有相同的类。

#### 最终生成的决策树
<img src="./decisionTrees.svg" />

#### 其它

如果数据集已经处理了所有属性，但是类标签依然不是惟一，即叶子节点还是可以再分的，此时我们需要决定如何定义该叶子节点，在这种情况下通常会采用多数表决的方法决定该叶子节点的分类。

后继还会介绍其它决策树算法，如 C4.5 和 CART，这些算法并不总是在每次划分分组时都会消耗特征。


### 算法实现

In [13]:
'''
决策树算法实现
'''
from numpy import *
from math import log
import operator


# 计算香农熵
def calcShannonEntropy(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for vec in dataSet:
        label = vec[-1]
        labelCounts[label] = labelCounts.get(label, 0) + 1
    
    shannonEntropy = 0.0
    for label in labelCounts:
        probability = float(labelCounts[label]) / numEntries
        shannonEntropy -= probability * log(probability, 2)
    return shannonEntropy

# 根据特征划分数据集
def splitDataSet(dataSet, fAxis, value):
    retDat = []
    for vec in dataSet:
        if vec[fAxis] == value:
            tmp = vec.copy() # 等价于 tmp = vec[:fAxis].extend(vec[fAxis+1:])
            del tmp[fAxis]
            retDat.append(tmp)
    return retDat

# 选择最优的数据集划分特征
def chooseBestFeature(dataSet):
    numDs = float(len(dataSet))
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEntropy(dataSet)
    bestInfoGain = 0.0; bestFeature = -1
    for i in range(numFeatures):
        values = [example[i] for example in dataSet]
        valuesSet = set(values)
        newEntropy = 0.0
        for v in valuesSet:
            subDataV = splitDataSet(dataSet, i, v)
            prop = float(len(subDataV)) / numDs
            newEntropy += prop * calcShannonEntropy(subDataV)
            
        infoGain = baseEntropy - newEntropy
        print ('feature ', i, 'infoGain: ', infoGain)
        if infoGain >= bestInfoGain:
            bestFeature = i
            bestInfoGain = infoGain
    return bestFeature

# 对于未能完全划分的叶子节点根据投票来获取分类
def cleafMajorityCount(leafList):
    labelCount = {}
    for vote in leafList:
        labelCount[vote] = labelCount.get(vote, 0) + 1
    sortedCount = sorted(labelCount.items(), key = operator.itemgetter(1), reverse = True)
    return sortedCount[0][0]

# 递归创建决策树
def createDecisionTree(dataSet, labels):
    classList = [example[-1]  for example in dataSet]
    # 如果类别完全相同则停止继续划分
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    # 如果没有特征可用时，用投票算法来完成分类。
    if len(dataSet[0]) == 1:
        return cleafMajorityCount(classList)
    
    bestFeature = chooseBestFeature(dataSet)
    bestFeatureLabel = labels[bestFeature]
    myTree = {bestFeatureLabel: {}}
    del labels[bestFeature]
    values = [example[bestFeature] for example in dataSet]
    valuesSet = set(values)
    for value in valuesSet:
        subLabels = labels[:]
        myTree[bestFeatureLabel][value] = createDecisionTree(splitDataSet(dataSet, bestFeature, value), subLabels)
    return myTree
    
            
# 创建数据集
def createDataSet():
    dataSet = [
        [1, 1, 'yes'],
        [1, 1, 'yes'],
        [1, 0, 'no'],
        [0, 1, 'no'],
        [0, 1, 'no']
    ]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels

In [124]:
import copy

# 熵函数测试
myDat, labels = createDataSet()
print (myDat)
print ('Initial Shannon Entropy: ', calcShannonEntropy(myDat), '\n')
'''
熵越高，则混合的数据也就越多，我们可以在数据集增加新的分类，观察熵的变化，这里增加一个新的 'maybe' 分类。
'''
testDat = copy.deepcopy(myDat)
testDat[0][-1] = 'maybe'
print (testDat)
print ('Add another label', calcShannonEntropy(testDat))


[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
Initial Shannon Entropy:  0.9709505944546686 

[[1, 1, 'maybe'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
Add another label 1.3709505944546687


In [125]:
# 测试根据特征划分数据集函数 
print ('Split by feature 0(no surfacing)')
print(splitDataSet(myDat, 0, 0), '\t', splitDataSet(myDat, 0, 1))
print ('Split by feature 1(flippers)')
print(splitDataSet(myDat, 1, 0), '\t', splitDataSet(myDat, 1, 1))

Split by feature 0(no surfacing)
[[1, 'no'], [1, 'no']] 	 [[1, 'yes'], [1, 'yes'], [0, 'no']]
Split by feature 1(flippers)
[[1, 'no']] 	 [[1, 'yes'], [1, 'yes'], [0, 'no'], [0, 'no']]


In [126]:
# 测试选择最优的划分特征
print (myDat)
print ('Initial Choose:', chooseBestFeature(myDat), '\n')
print (splitDataSet(myDat, 0, 0))
print ('Another Choose:',chooseBestFeature(splitDataSet(myDat, 0, 0)))

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
feature  0 infoGain:  0.4199730940219749
feature  1 infoGain:  0.17095059445466854
Initial Choose: 0 

[[1, 'no'], [1, 'no']]
feature  0 infoGain:  0.0
Another Choose: 0


In [127]:
# test cleafMajorityCount
cleafMajorityCount(['y','y','n'])

'y'

In [14]:
# 获取最终的决策树
myDat, labels = [[1, 'yes'], [1, 'yes'], [0, 'no']], ['flippers']
print (myDat, labels)
print (createDecisionTree(myDat, labels))
print ()
myDat, labels = [[1, 'yes'], [1, 'yes']], ['flippers']
print (myDat, labels)
print (createDecisionTree(myDat, labels))
print ()
myDat, labels = createDataSet()
createDecisionTree(myDat, labels)


[[1, 'yes'], [1, 'yes'], [0, 'no']] ['flippers']
feature  0 infoGain:  0.9182958340544896
{'flippers': {0: 'no', 1: 'yes'}}

[[1, 'yes'], [1, 'yes']] ['flippers']
yes

feature  0 infoGain:  0.4199730940219749
feature  1 infoGain:  0.17095059445466854
feature  0 infoGain:  0.9182958340544896


{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

### 使用 Matplotlib 绘制决策树
```javascript
// 获取树的深度 js 版实现
var tree = {'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
var getTreeDepth = function(tree){
  if (tree && typeof tree == 'object' &&  Object.keys(tree).length > 0) {
    var keys = Object.keys(tree)
    var subTree = tree[keys[0]]
    var subKeys = Object.keys(subTree)
    var leftDepth = 1 +  getTreeDepth(subTree[subKeys[0]])
    var rightDepth = 1 +  getTreeDepth(subTree[subKeys[1]])
    return leftDepth > rightDepth ? leftDepth : rightDepth
  }
  return 0
}
getTreeDepth(tree)
```

In [2]:
'''
绘制决策树
'''
import matplotlib.pyplot as plt

# 获取树的深度
def getTreeDepth(tree):
    if tree == None or (not isinstance(tree, dict)) or len(tree.keys()) == 0:
        return 0
    subTreeKey = list(tree)[0]
    subTree = tree[subTreeKey]
    subTreeKeys = subTree.keys()
    maxSubTreeDepth = 0
    for subKey in subTreeKeys:
        depth = getTreeDepth(subTree[subKey])
        if depth >= maxSubTreeDepth:
            maxSubTreeDepth = depth
    return 1 + maxSubTreeDepth

# 获取树的最大宽度，即叶子节点个数。
def getTreeWidth(tree):
    if tree == None:
        return 0
    if isinstance(tree, str):
        return 1
    keys = list(tree.keys())
    subTree = tree[keys[0]]
    subTreeKeys = list(subTree)
    numLeaves = 0
    for subKey in subTreeKeys:
        numLeaves += getTreeWidth(subTree[subKey])
    return numLeaves

# 判断是否是叶子节点
def isLeaf(node):
    return isinstance(node, dict) and len(node.keys()) == 1 and isinstance(node[list(node)[0]], str)

# 获取树的根节点位置
def getRootPos(tree, xStartPos = 0):
    width = getTreeWidth(tree)
    height = getTreeDepth(tree)
    return (width - 1) * 4 + xStartPos, height * 2

# 绘制树的节点
def plotTreeNodes(tree, xStartPos = 0):
    if isLeaf(tree):
        print(xStartPos + 2, 2)
        return
    xPos, yPos = getRootPos(tree)
    keys = list(tree.keys())
    rootTree = tree[keys[0]]
    subKeys = list(rootTree)
    xStartPos = 0
    for subKey in subKeys:
        print ('subTree:', rootTree, subKey)
        subTree = rootTree[subKey]
        plotTreeNodes(subTree, xStartPos)
        xStartPos, y = getRootPos(subTree)
    print (xPos, yPos)


In [3]:
tree1 = {'no surfacing': {0: 'no', 1: {'flippers': {0: {'flippers': {0: 'no', 1: 'yes'}}, 1: 'yes'}}}}
tree2 = {'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
tree3 = {'no surfacing': {0: 'no', 1: 'no', 2: 'yes'}}
print(getTreeDepth(tree1), getTreeDepth(tree2), getTreeDepth(tree3))
print(getTreeWidth(tree1), getTreeWidth(tree2), getTreeWidth(tree3))
print(isLeaf({0: 'yes'}), isLeaf(tree3))
print(getRootPos(tree1), getRootPos(tree2), getRootPos(tree3))
print ('Plot tree')
plotTreeNodes(tree1)

3 2 1
4 3 3
True False
(12, 6) (8, 4) (8, 2)
Plot tree
subTree: {0: 'no', 1: {'flippers': {0: {'flippers': {0: 'no', 1: 'yes'}}, 1: 'yes'}}} 0


AttributeError: 'str' object has no attribute 'keys'